In [14]:
from src.cleanfunctions import *
from src.geofunctions import *
from src.apiCalls import *
from pymongo import MongoClient
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster,FastMarkerCluster

In [15]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
company_processed = clean(db)


# Near successful tech(raised>1M)

QUERIES
Developers like to be near successful tech startups that have raised at least 1 Million dollars.

In [65]:
raised=list(db.companies_processed.find({"total_money_raised":{"$regex":"M"}},{"_id":0}))
raisedDF=pd.DataFrame(raised)
raisedDF.head(5)

,name,category_code,office,clean_state,city,country_code,latitude,longitude,total_money_raised
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success,Seattle,USA,47.603122,-122.333253,$39.8M
1,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,...",success,San Francisco,USA,37.764726,-122.394523,$45M
2,Slacker,music,"{'type': 'Point', 'coordinates': [-117.081406,...",success,San Diego,USA,33.022176,-117.081406,$73.1M
3,Gizmoz,web,"{'type': 'Point', 'coordinates': [-122.169472,...",success,Menlo Park,USA,37.484130,-122.169472,$18.1M
4,Lala,games_video,"{'type': 'Point', 'coordinates': [-122.154369,...",success,Palo Alto,USA,37.451151,-122.154369,$44.2M


In [17]:
raisedDF = raisedDF[raisedDF['city'].isin(raisedDF['city'].value_counts()[raisedDF['city'].value_counts() >200].index)]
raisedDF.city.value_counts()

San Francisco    276
Name: city, dtype: int64

In [18]:
raised1M=raisedDF.to_json("OUTPUT/raisedMillon.json",orient="records")

In [19]:
coordCity = geocode("San Francisco USA")

APIKEY FOUND!!
<Response [200]>


In [20]:
mapcoord =coordCity["coordinates"]

In [21]:
foliumMap = folium.Map(location=mapcoord)

In [22]:
foliumMap

In [23]:
million = pd.read_json("OUTPUT/raisedMillon.json")

In [24]:
million=million.drop(["office"], axis=1)
rM= folium.FeatureGroup(name='raisedMillion')
FastMarkerCluster(million[['latitude', 'longitude']]).add_to(rM)
rM.add_to(foliumMap)
foliumMap

# Airports

QUERIES Account managers need to travel a lot. Search place with airports


In [25]:
airportSF = getAirport("San Francisco")


In [26]:
airportSF =airportSF["response"]
airportSF =airportSF["venues"]

In [50]:
name = [(e["name"]) for e in airportSF]
loc = [(e["location"]) for e in airportSF]

In [51]:
lat=[]
lng=[]
for e in loc:
    lat.append(e["lat"])
    lng.append(e["lng"])

In [52]:
airports= pd.DataFrame(name)
airports["name"]=name
airports["latitude"]=lat
airports["longitude"]=lng
airports=airports.drop([0], axis=1)

In [53]:
airports=pd.DataFrame(airports).to_json("OUTPUT/airports.json",orient="records")

In [54]:
airFol = pd.read_json("OUTPUT/airports.json")

In [55]:
aF= folium.FeatureGroup(name='airports')
MarkerCluster(airFol[['latitude', 'longitude']],popup="AIRPORT").add_to(aF)
aF.add_to(foliumMap)
foliumMap

# Starbucks

In [56]:
starbucks= getStarbucks("San Francisco")

In [57]:
starbucks =starbucks["response"]
starbucks =starbucks["venues"]

In [58]:
name = [(e["name"]) for e in starbucks]
loc = [(e["location"]) for e in starbucks]

In [59]:
lat=[]
lng=[]
for e in loc:
    lat.append(e["lat"])
    lng.append(e["lng"])

In [60]:
starbucks= pd.DataFrame(name)
starbucks["name"]=name
starbucks["latitude"]=lat
starbucks["longitude"]=lng
starbucks = starbucks.drop([0], axis=1)

In [61]:
starbucks=pd.DataFrame(starbucks).to_json("OUTPUT/starbucks.json",orient="records")


In [62]:
starbucksF = pd.read_json("OUTPUT/starbucks.json")

In [63]:
sBucks= folium.FeatureGroup(name='Starbucks')
HeatMap(starbucksF[['latitude', 'longitude']]).add_to(sBucks)
sBucks.add_to(foliumMap)
foliumMap

In [41]:
foliumMap.save("OUTPUT/index.html")

# Lugar elegido

Veo que los datos que he recogido según los mapas,por lo que he ordenado el dataframe de las oficinas más exitosas y escogido su mediana,para ver en que coordenadas se concentran más los datos. Los Starbucks se concentran en latitud y longitud parecidas, por lo que el punto escogido lo cojo por la media entre ambos puntos.

In [42]:
sortByLatM = million.sort_values('latitude')
medianM =sortByLatM.median()
print("Median:\n",medianM)

Median:
 latitude      37.783171
longitude   -122.400922
dtype: float64


In [43]:
sortByLatS = starbucksF.sort_values('latitude')
medianS =sortByLatS.median()
print("Median:\n",medianS)

Median:
 latitude      37.785387
longitude   -122.409748
dtype: float64


In [44]:
latResult=(medianM.latitude+medianS.latitude)/2
lonResult=(medianM.longitude+medianS.longitude)/2
result=[latResult,lonResult]
result

[37.784278919325004, -122.40533450000001]

In [48]:
folium.Marker(result, popup='RESULT', tooltip="result").add_to(foliumMap)
foliumMap

Tiene un StarBucks a menos de 200m y un aeropuerto a menos de 20km

In [46]:
getNear(result[0],result[1],200,"Starbucks")

{'meta': {'code': 200, 'requestId': '5e9e904547e0d6002826292d'},
 'response': {'venues': [{'id': '57a0e506498e6c087e114f97',
    'name': 'Starbucks',
    'location': {'address': '865 Market St, C 26A',
     'crossStreet': '5th street',
     'lat': 37.78414314368152,
     'lng': -122.40759611781183,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.78414314368152,
       'lng': -122.40759611781183}],
     'distance': 199,
     'postalCode': '94103',
     'cc': 'US',
     'city': 'San Francisco',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['865 Market St, C 26A (5th street)',
      'San Francisco, CA 94103',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15874

In [47]:
getNear(result[0],result[1],20000,"International Airport")

{'meta': {'code': 200, 'requestId': '5e9e91365fb726001b5c093b'},
 'response': {'venues': [{'id': '41059b00f964a520850b1fe3',
    'name': 'San Francisco International Airport (SFO) (San Francisco International Airport)',
    'location': {'address': 'N McDonnell Rd',
     'crossStreet': 'at Link Rd',
     'lat': 37.6167130000997,
     'lng': -122.38709449768066,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.6167130000997,
       'lng': -122.38709449768066}],
     'distance': 18722,
     'postalCode': '94128',
     'cc': 'US',
     'city': 'San Francisco',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['N McDonnell Rd (at Link Rd)',
      'San Francisco, CA 94128',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ed931735',
      'name': 'Airport',
      'pluralName': 'Airports',
      'shortName': 'Airport',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
       'suffix': '.png'},
      